In [ ]:
!git clone https://github.com/DrikDrik/sber-movqgan-project

In [ ]:
import sys
sys.path.append('/content/sber-movqgan-project/')

In [ ]:
!pip install pip==24.0
!pip install -r 'sber-movqgan-project/requirements.txt'

In [ ]:
from models.RealNVP.model import RealNVP
from models.RealNVP.train import training_loop
from pipeline.movqgan import decode
from pipeline.show import show_images

In [ ]:
from utils.imports import *

In [ ]:
from pipeline.movqgan import get_model, decode
model = get_model()

In [ ]:
from utils import download_datasets
download_datasets.main()

In [ ]:
!unzip -q datasets/latent_dataset.zip -d '/content/latent_representations/'

In [ ]:
from pipeline.get_latent import LatentDataset

In [ ]:
import os
temp_dataset = LatentDataset('latent_representations')
temp_loader = DataLoader(temp_dataset, batch_size=128, shuffle=True)

In [ ]:
from utils import normalize
normalize_fn = normalize.normalize_fn(temp_loader)
inverse_normalize_fn = normalize.inverse_normalize_fn(temp_loader)

In [ ]:
transform = transforms.Compose([
    transforms.Lambda(normalize_fn)
])
dataset = LatentDataset('latent_representations', transform=transform)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
store_path = "/content/checkpoint/project_nvp.ckpt"

In [ ]:
# Training
device='cuda'
nvp = RealNVP().to(device)
n_epochs = 200
lr = 0.00003
if not os.path.isdir(os.path.dirname(store_path)): os.mkdir(os.path.dirname(store_path))
training_loop(nvp, loader, n_epochs, optim=torch.optim.AdamW(nvp.parameters(), lr), device=device, model=model, inverse_normalize_fn=inverse_normalize_fn, store_path=store_path)

